In [1]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [2]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [3]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2023-01-05 08:32:23,346 INFO: ====== WebDriver manager ======
2023-01-05 08:32:23,678 INFO: Get LATEST geckodriver version for 108.0 firefox


[WDM] - Downloading: 19.0kB [00:00, 9.89MB/s]                   


2023-01-05 08:32:24,217 INFO: Driver [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32\geckodriver.exe] found in cache


**Scrape New Completed Games and Format Them**

In [4]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 52 entries, 0 to 51
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            52 non-null     int64         
 1   GAME_DATE_EST   52 non-null     datetime64[ns]
 2   HOME_TEAM_WINS  52 non-null     int64         
 3   PTS             52 non-null     int64         
 4   FG_PCT          52 non-null     float64       
 5   FG3_PCT         52 non-null     float64       
 6   FT_PCT          52 non-null     float64       
 7   REB             52 non-null     int64         
 8   AST             52 non-null     int64         
 9   TEAM_ID         52 non-null     object        
 10  GAME_ID         52 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 4.9+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-04,1,131,50.0,30.0,75.7,59,28,1610612763,22200564,107,40.4,28.9,83.3,44,25,1610612766,2022
1,2023-01-04,0,88,39.5,35.7,77.8,49,24,1610612756,22200565,90,39.0,30.6,82.6,46,15,1610612739,2022
2,2023-01-04,0,115,40.4,40.5,90.3,33,23,1610612760,22200566,126,51.9,39.4,80.6,49,26,1610612753,2022
3,2023-01-04,0,126,47.4,36.1,88.5,45,31,1610612754,22200567,129,49.0,38.9,86.4,46,24,1610612755,2022
4,2023-01-04,0,114,50.0,39.3,80.8,38,25,1610612759,22200568,117,47.9,32.3,65.4,47,16,1610612752,2022
5,2023-01-04,0,108,44.3,31.6,66.7,43,25,1610612745,22200572,119,46.1,36.8,88.5,51,25,1610612740,2022
6,2023-01-04,1,104,39.8,33.3,70.0,73,26,1610612749,22200569,101,32.8,19.6,80.0,47,22,1610612761,2022
7,2023-01-04,0,112,46.2,32.4,76.2,34,24,1610612751,22200570,121,53.1,40.0,93.1,46,21,1610612741,2022
8,2023-01-04,0,106,44.0,31.0,73.8,39,18,1610612757,22200571,113,51.3,37.5,72.2,33,25,1610612750,2022
9,2023-01-04,1,120,53.6,33.3,90.9,44,21,1610612737,22200575,117,46.6,40.0,92.6,42,26,1610612758,2022


**Retrieve todays games**

In [5]:
#retrieve list of teams playing today

# get today's games on NBA schedule
matchups, game_ids = get_todays_matchups(driver)


print(matchups)
print(game_ids)


[['1610612763', '1610612766'], ['1610612756', '1610612739'], ['1610612760', '1610612753'], ['1610612754', '1610612755'], ['1610612759', '1610612752'], ['1610612749', '1610612761'], ['1610612751', '1610612741'], ['1610612757', '1610612750'], ['1610612745', '1610612740'], ['1610612765', '1610612744'], ['1610612748', '1610612747'], ['1610612737', '1610612758']]
['22200564', '22200565', '22200566', '22200567', '22200568', '22200569', '22200570', '22200571', '22200572', '22200573', '22200574', '22200575']


**Close Webdriver**

In [6]:
driver.close() 

**Create Rows for Today's Games with Empty Stats**

In [7]:
# append today's matchups to the new games dataframe


df_today = df_new.drop(df_new.index) #empty copy of df_new with same columns
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': int(game_ids[i]),                       
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612766,22200564,0.0,0.0,0.0,0.0,0.0,0.0,1610612763,2022
1,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612739,22200565,0.0,0.0,0.0,0.0,0.0,0.0,1610612756,2022
2,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612753,22200566,0.0,0.0,0.0,0.0,0.0,0.0,1610612760,2022
3,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612755,22200567,0.0,0.0,0.0,0.0,0.0,0.0,1610612754,2022
4,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612752,22200568,0.0,0.0,0.0,0.0,0.0,0.0,1610612759,2022
5,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612761,22200569,0.0,0.0,0.0,0.0,0.0,0.0,1610612749,2022
6,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612741,22200570,0.0,0.0,0.0,0.0,0.0,0.0,1610612751,2022
7,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612750,22200571,0.0,0.0,0.0,0.0,0.0,0.0,1610612757,2022
8,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612740,22200572,0.0,0.0,0.0,0.0,0.0,0.0,1610612745,2022
9,2023-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1610612744,22200573,0.0,0.0,0.0,0.0,0.0,0.0,1610612765,2022


**Access Feature Store**

In [8]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350
Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [9]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [10]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2023-01-05 08:35:56,208 INFO: USE `nba_predictor_featurestore`
2023-01-05 08:35:56,615 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,2005-02-12,20400743,1610612765,1610612764,2004,107,0.532227,0.738770,0.500000,33,42,86,0.363037,0.666992,0.399902,19,46,1
23312,2018-10-29,21800092,1610612749,1610612761,2018,124,0.489014,0.789062,0.422119,28,57,109,0.378906,0.786133,0.199951,25,51,1
23313,2011-01-24,21000653,1610612751,1610612739,2010,103,0.446045,0.758789,0.538086,24,40,101,0.451904,0.833008,0.356934,18,41,1
23314,2017-03-06,21600934,1610612755,1610612749,2016,98,0.437988,0.631836,0.267090,30,42,112,0.493896,0.799805,0.424072,28,41,0


**Convert Feature Names back to original mixed case**

In [11]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2004-11-17,20400109,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,2005-02-12,20400743,1610612765,1610612764,2004,107,0.532227,0.738770,0.500000,33,42,86,0.363037,0.666992,0.399902,19,46,1
23312,2018-10-29,21800092,1610612749,1610612761,2018,124,0.489014,0.789062,0.422119,28,57,109,0.378906,0.786133,0.199951,25,51,1
23313,2011-01-24,21000653,1610612751,1610612739,2010,103,0.446045,0.758789,0.538086,24,40,101,0.451904,0.833008,0.356934,18,41,1
23314,2017-03-06,21600934,1610612755,1610612749,2016,98,0.437988,0.631836,0.267090,30,42,112,0.493896,0.799805,0.424072,28,41,0


**Update Yesterday's Matchup Predictions with New Final Results**

In [12]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results


df_old = df_old.set_index('GAME_ID')
df_old.update(df_new.set_index('GAME_ID'))
df_old = df_old.reset_index()  


df_old

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,21700374,2017-12-08,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,21200874,2013-03-01,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,20500210,2005-11-30,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,21800395,2018-12-10,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,20400109,2004-11-17,1610612751,1610612760,2004,68,0.413086,0.889160,0.000000,9,37,79,0.406006,0.758789,0.333008,12,33,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23311,20400743,2005-02-12,1610612765,1610612764,2004,107,0.532227,0.738770,0.500000,33,42,86,0.363037,0.666992,0.399902,19,46,1
23312,21800092,2018-10-29,1610612749,1610612761,2018,124,0.489014,0.789062,0.422119,28,57,109,0.378906,0.786133,0.199951,25,51,1
23313,21000653,2011-01-24,1610612751,1610612739,2010,103,0.446045,0.758789,0.538086,24,40,101,0.451904,0.833008,0.356934,18,41,1
23314,21600934,2017-03-06,1610612755,1610612749,2016,98,0.437988,0.631836,0.267090,30,42,112,0.493896,0.799805,0.424072,28,41,0


**Add Today's Matchups for Feature Engineering**

In [13]:
df_combined = pd.concat([df_old, df_today], ignore_index = True)
df_combined

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,21700374,2017-12-08 00:00:00,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0
1,21200874,2013-03-01 00:00:00,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0
2,20500210,2005-11-30 00:00:00,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0
3,21800395,2018-12-10 00:00:00,1610612749,1610612739,2018,108.0,0.437988,0.817871,0.416992,22.0,58.0,92.0,0.375000,0.666992,0.333008,24.0,46.0,1.0
4,20400109,2004-11-17 00:00:00,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23323,22200571,2023-01-05,1610612750,1610612757,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23324,22200572,2023-01-05,1610612740,1610612745,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23325,22200573,2023-01-05,1610612744,1610612765,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
23326,22200574,2023-01-05,1610612747,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0


**Data Processing**

In [14]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_ID,GAME_DATE_EST,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS,PLAYOFF,TARGET
0,21700374,2017-12-08 00:00:00,1610612759,1610612738,2017,105.0,0.468994,0.875000,0.295898,16.0,46.0,102.0,0.458008,0.881836,0.289062,14.0,39.0,1.0,0,1.0
1,21200874,2013-03-01 00:00:00,1610612756,1610612737,2012,92.0,0.444092,0.833008,0.455078,16.0,38.0,87.0,0.425049,0.772949,0.347900,21.0,43.0,1.0,0,1.0
2,20500210,2005-11-30 00:00:00,1610612738,1610612755,2005,110.0,0.447998,0.784180,0.250000,24.0,59.0,103.0,0.408936,0.770996,0.308105,21.0,40.0,1.0,0,1.0
3,21800395,2018-12-10 00:00:00,1610612749,1610612739,2018,108.0,0.437988,0.817871,0.416992,22.0,58.0,92.0,0.375000,0.666992,0.333008,24.0,46.0,1.0,0,1.0
4,20400109,2004-11-17 00:00:00,1610612751,1610612760,2004,68.0,0.413086,0.889160,0.000000,9.0,37.0,79.0,0.406006,0.758789,0.333008,12.0,33.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23323,22200571,2023-01-05,1610612750,1610612757,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23324,22200572,2023-01-05,1610612740,1610612745,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23325,22200573,2023-01-05,1610612744,1610612765,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0
23326,22200574,2023-01-05,1610612747,1610612748,2022,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0,0.0


**Feature Engineering**

In [15]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


#fix type conversion issues with hopsworks
df_combined['TARGET'] = df_combined['TARGET'].astype('int16')
df_combined['HOME_TEAM_WINS'] = df_combined['HOME_TEAM_WINS'].astype('int16')


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

**Insert New Data into Feature Group**

In [16]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/23328 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x20733289760>, None)